In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.models import Sequential
import numpy as np 


: 

In [ ]:

# Define the input shape
input_shape = (30, 31, 1)  # Grayscale images have a single channel
num_classes = 3
model = Sequential()

# First Convolutional Layer
model.add(Input(input_shape))
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
# Second Convolutional Layer
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
# Flatten the 2D arrays for the fully connected layers
model.add(Flatten())
# Fully connected layer
model.add(Dense(units=128, activation='relu'))
# Dropout for regularization
model.add(Dropout(0.3))
# Output layer
model.add(Dense(units=num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

: 

In [3]:
# Summary of the model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 29, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 7, 7, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 5, 5, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 1, 1, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,523 (107.51 KB)

 Trainable params: 27,523 (107.51 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# Directory containing the dataset
data_dir = './dataset'

# Load the dataset
batch_size = 32
img_height = 31
img_width = 30

# Load training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',  # can be 'int' for integer labels or 'categorical' for one-hot encoded labels
    color_mode='grayscale',  # because the images are grayscale
    batch_size=batch_size,
    image_size=(img_height, img_width),  # resize the images
    shuffle=True,
    seed=42,
    validation_split=0.1,
    subset='training',
)

# Load validation dataset
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=42,
    validation_split=0.1,
    subset='validation',
)

Found 66 files belonging to 3 classes.
Using 60 files for training.
Found 66 files belonging to 3 classes.
Using 6 files for validation.


In [5]:
def normalize(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_dataset = train_dataset.map(normalize)
validation_dataset = validation_dataset.map(normalize)
train_dataset = train_dataset.repeat(10)
validation_dataset = validation_dataset.repeat(10)

In [150]:
def random_column_cutoff(image, label):
    # Get the batch size, height, and width of the image
    batch_size = tf.shape(image)[0]
    img_height = tf.shape(image)[1]
    img_width = tf.shape(image)[2]
    
    # Randomly choose a cutoff N (between 0 and img_width)
    N = tf.random.uniform([], minval=0, maxval=img_width, dtype=tf.int32)
    
    # Create a mask with ones up to column N and zeros for columns > N
    mask = tf.concat([
        tf.ones((batch_size, img_height, N, 1)), 
        tf.zeros((batch_size, img_height, img_width - N, 1))
    ], axis=2)
    
    # Apply the mask to the image
    augmented_image = image * mask
    
    return augmented_image, label

In [151]:
train_dataset = train_dataset.map(lambda x, y: random_column_cutoff(x, y))
validation_dataset = validation_dataset.map(lambda x, y: random_column_cutoff(x, y))

In [15]:
model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=12
)

Epoch 1/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9162 - loss: 0.1859 - val_accuracy: 1.0000 - val_loss: 0.0863
Epoch 2/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9290 - loss: 0.1775 - val_accuracy: 1.0000 - val_loss: 0.1073


In [20]:
model.save('./model.h5')

INFO:tensorflow:Assets written to: ./model/assets


INFO:tensorflow:Assets written to: ./model/assets


Saved artifact at './model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 30, 31, 1), dtype=tf.float32, name='keras_tensor_74')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  134598071260912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134598060371008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134598060375056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134598060375760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134598060746080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134598060746960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134598060934592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134598060934064: TensorSpec(shape=(), dtype=tf.resource, name=None)
